# Sensor Anomaly Model Serving

In this notebook we quickly explore some specific aspects of the multi-level state management in Spark Streaming

By combining local and distributed state, we implement a simple sensor trend tracker that can help us identify and report anomalies.

## Our Streaming dataset will consist of sensor information, containing the sensorId, a timestamp, and a value.
This component is a participant in a streaming pipeline.

It expects to receive moving averages of sensor data in the form of (id, timestamp, value) 

In [ ]:
import org.apache.spark.streaming.Seconds
val topic = "sensor-processed"
val kafkaBootstrapServer = "172.17.0.2:9092"
val modelTopic = "modelTopic"
val interval = Seconds(10)

import org.apache.spark.streaming.Seconds
topic: String = sensor-processed
kafkaBootstrapServer: String = 172.17.0.2:9092
modelTopic: String = modelTopic
interval: org.apache.spark.streaming.Duration = 10000 ms


# Create a Streaming Standard Deviation Model
## We can't use the classical model, as the data "on the move".
The classical formula for standard deviation : ![Standard Deviation Equation](https://wikimedia.org/api/rest_v1/media/math/render/svg/00eb0cde84f0a838a2de6db9f382866427aeb3bf) requires that all data is known beforehand.

##  We need a specialized streaming algorithm
To obtain the standard deviation of a streaming dataset, we need an algorithm that provides an approximation of the `stdev` value.

Based on https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance  we've chosen to implement [Welford's Online algorithm](https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance#Welford's_Online_algorithm)

In [ ]:
// State Definition for keeping the number of samples (n), the mean and the aggregated squared distance from the mean (m2)
case class M2(n:Int, mean: Double, m2:Double) {
  def variance: Option[Double] = {
    if (n<2) None else Some(m2/(n-1))
  }
  def stdev: Option[Double] = variance.map(Math.sqrt)
  }
  object M2 extends Serializable {
    val Zero = M2(0, 0.0, 0.0)
  }

defined class M2
defined object M2


In [ ]:
// A collection to hold the state of all identified elements
// this needs to be outside of the class b/c of Spark Notebook serialization
var entries:Map[String, M2] = Map.empty

entries: Map[String,M2] = Map()


## Welford's Algorithm Implemented on Spark Streaming
We use the low-level capabilities of Spark Streaming to implement a distributed computation of the updated values for
each element in the incoming stream.
We combine that with a local in-memory structure to keep only the computed parameters for each entry.

Note how computations on `RDD`s are executed in a distributed context. Only the results are collected back in the local context.
We define two methods: 
- `trainOn`: Receives a distributed collection and updates the model parameters for each id
- `predictOnValues`: Estimates the mean and standard deviation for a given id.

In [ ]:
import org.apache.spark.rdd.RDD
import org.apache.spark.SparkContext
import org.apache.spark.streaming.dstream.DStream
  
class M2Model() extends Serializable {
  
  def trainOn(dstream: DStream[(String, Double)]): Unit = {
    dstream.foreachRDD{rdd => 
                       // This computation executes distributedly
                       val newEntriesRDD = rdd.map{case (id, x) => 
                                                val current = entries.get(id)
                                                val updated = current.map{case M2(n, mean, m2) => {
                                                  val np = n + 1
                                                  val delta = x - mean
                                                  val meanp = mean + delta/np
                                                  val mp2 = m2 + delta*(x - meanp)
                                                  (id, M2(np, meanp, mp2))
                                                  }
                                                 }.getOrElse(id -> M2.Zero)
                                                 updated
                                               }
                       // Here, we collect the results and update the local state
                       val newEntries: Array[(String, M2)] = newEntriesRDD.collect
                       entries = entries ++ newEntries
                      }
  }
  
  def predictOnValues(dstream: DStream[(String, Double)]): DStream[(String, Double, Double, Double)] = {
    for { 
      (id, value) <- dstream
      m2 <- entries.get(id)
      stdev <- m2.stdev
    } yield (id, value, m2.mean, stdev)
  }
}

<console>:54: warning: `withFilter' method does not yet exist on org.apache.spark.streaming.dstream.DStream[(String, Double)], using `filter' method instead
             (id, value) <- dstream
                            ^
import org.apache.spark.rdd.RDD
import org.apache.spark.SparkContext
import org.apache.spark.streaming.dstream.DStream
defined class M2Model


## We create our Streaming Context

In [ ]:
import org.apache.spark.streaming.StreamingContext
@transient val streamingContext = new StreamingContext(sparkSession.sparkContext, interval)

import org.apache.spark.streaming.StreamingContext
streamingContext: org.apache.spark.streaming.StreamingContext = org.apache.spark.streaming.StreamingContext@1d81accd


## Our stream source will be a a Direct Kafka Stream


In [ ]:
import org.apache.kafka.clients.consumer.ConsumerRecord
import kafka.serializer.StringDecoder
import org.apache.spark.streaming.kafka._

val kafkaParams = Map[String, String](
  "metadata.broker.list" -> kafkaBootstrapServer,
  "group.id" -> "sensor-tracker-group",
  "auto.offset.reset" -> "largest"
)

val topics = Set(topic)
@transient val stream = KafkaUtils.createDirectStream[String, String, StringDecoder, StringDecoder](
     streamingContext, kafkaParams, topics)

// kafka_010 APIs don't work on the Spark Notebook

// @transient val stream = KafkaUtils.createDirectStream[String, String](
//   streamingContext,
//   PreferConsistent,
//   Subscribe[String, String](topics, kafkaParams)
// )



<console>:81: warning: object KafkaUtils in package kafka is deprecated: Update to Kafka 0.10 integration
       @transient val stream = KafkaUtils.createDirectStream[String, String, StringDecoder, StringDecoder](
                               ^
import org.apache.kafka.clients.consumer.ConsumerRecord
import kafka.serializer.StringDecoder
import org.apache.spark.streaming.kafka._
kafkaParams: scala.collection.immutable.Map[String,String] = Map(metadata.broker.list -> 172.17.0.2:9092, group.id -> sensor-tracker-group, auto.offset.reset -> largest)
topics: scala.collection.immutable.Set[String] = Set(sensor-processed)
stream: org.apache.spark.streaming.dstream.InputDStream[(String, String)] = org.apache.spark.streaming.kafka.DirectKafkaInputDStream@60d44806


# Providing Schema information for our streaming data
Now that we have a DStream of fresh data processed in a 2-second interval, we can start focusing on the gist of this example.
First, we want to define and apply a schema to the data we are receiving.
In Scala, we can define a schema with a `case class`

In [ ]:
case class SensorData(id: String, timestamp: Long, value: Double)

defined class SensorData


# Instantiate our Model
We will train an online standard deviation algorithm and use it to score the incoming data.

In [ ]:
val model = new M2Model()

model: M2Model = M2Model@29e35041


# Convert the incoming JSON to `SensorData`
See how we interop with SparkSQL from Spark Streaming to use the JSON parsing facilities.

In [ ]:
val spark = sparkSession
import spark.implicits._
@transient val sensorDataStream = stream.transform{rdd => 
                                        val jsonData = rdd.map{case (k,v)  => v}
                                        val ds = sparkSession.createDataset(jsonData)
                                        val jsonDF = spark.read.json(ds)
                                        val sensorDataDS = jsonDF.as[SensorData]
                                        sensorDataDS.rdd
                                       }

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@44287f49
import spark.implicits._
sensorDataStream: org.apache.spark.streaming.dstream.DStream[SensorData] = org.apache.spark.streaming.dstream.TransformedDStream@1dd636f5


## Prepare the data to train our model
We require the Id and the value that we want to use in our online standard deviation

In [ ]:
@transient val inputData = sensorDataStream.transform {sensorDataRDD =>  
                                                       sensorDataRDD.map{case SensorData(id,ts,value) => (id, value)}}                                                            

inputData: org.apache.spark.streaming.dstream.DStream[(String, Double)] = org.apache.spark.streaming.dstream.TransformedDStream@7ea5b695


## Use the data to train the model

In [ ]:
model.trainOn(inputData)

## Score the streaming data using the trained model

In [ ]:
@transient val scored = model.predictOnValues(inputData)

scored: org.apache.spark.streaming.dstream.DStream[(String, Double, Double, Double)] = org.apache.spark.streaming.dstream.FlatMappedDStream@10826d8b


### Visualize the relation between the values and their standard deviation

In [ ]:
val scatterChart = new ScatterChart(Seq((0.0,0.0)))

scatterChart: notebook.front.widgets.charts.ScatterChart[Seq[(Double, Double)]] = <ScatterChart widget>


### Ouput Operations give us access to the data

In [ ]:
scored.foreachRDD{rdd =>
  val data = rdd.collect.map{case (id, value, mean, std) => (value, std)}.take(200)
  scatterChart.applyOn(data)
}

In [ ]:
// Show the chart
scatterChart

res17: notebook.front.widgets.charts.ScatterChart[Seq[(Double, Double)]] = <ScatterChart widget>


entries total

In [ ]:
// Declare UI Widgets to see the data
val debugBox = ul(15)
debugBox.append("---")

debugBox: notebook.front.widgets.HtmlList = <HtmlList widget>


### Stream Content log (for mental sanity purposes)

In [ ]:
debugBox

res21: notebook.front.widgets.HtmlList = <HtmlList widget>


## `foreachRDD` Stream Output
The output operation lets us materialize the results.

In this notebook, we are going to output the results to the UI widgets we declared earlier.

In [ ]:
inputData.foreachRDD{rdd => 
                    val sample = rdd.take(20).map(_.toString)
                    debugBox.appendAll(sample)
                   } 

## Write the trained model to the messaging backend

In [ ]:
case class IdM2(id:String, m2: M2)

defined class IdM2


In [ ]:
val spark = sparkSession
import spark.implicits._
import org.apache.spark.sql.types.StringType

inputData.window(Seconds(30)).foreachRDD{ (rdd,time) => 
                                         if (!rdd.isEmpty) {
                                           val modelDF = entries.map{case (id, m2) => IdM2(id, m2)}.toSeq.toDF
                                           .select(to_json(struct("*")).as("value"))
                                           .withColumn("key", lit(null).cast(StringType))
                                           modelDF.write.format("kafka")
                                           .option("kafka.bootstrap.servers", kafkaBootstrapServer)
                                            .option("topic", modelTopic)
                                           .save()
                                         }
                                        }

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@44287f49
import spark.implicits._
import org.apache.spark.sql.types.StringType


## Starting the Context  initiates the stream processing

In [ ]:
streamingContext.start()

## `stop` destroys the streamingContext and stops the streaming computation  

In [ ]:
// Be careful not to stop the context if you want the streaming process to continue
streamingContext.stop(false)

### We can 'snoop' in the values of our model. 
The values are local to this process. Only computing them is done distributedly. 

In [ ]:
entries("office").stdev

res30: Option[Double] = Some(10.835718130348406)


Some(10.835718130348406)

In [ ]:
entries("office").mean

res32: Double = 36.38722109533468


36.38722109533468

In [ ]:
entries("office")

res34: M2 = M2(18,38.15306513409961,3880.347227506937)


M2(18,38.15306513409961,3880.347227506937)